# MadMiner physics tutorial (part 1)

Johann Brehmer, Felix Kling, Irina Espejo, and Kyle Cranmer 2018-2019

Welcome to the MadMiner particle-physics tutorial! It consists of six numbered notebooks. They build on each other, so please go through them in the right order, starting with this notebook.

In this first part, we'll show how to set up the parameter space and define a morphing setups -- typically the first steps in a MadMiner analysis.

This tutorial does not try to explain the theory behind the tool. To understand what MadMiner is doing, please have a look at some papers first. In
["Constraining Effective Field Theories With Machine Learning"](https://arxiv.org/abs/1805.00013)
we explain the basic idea of most of the methods presented here, while ["A Guide to Constraining Effective Field Theories With Machine Learning"](https://arxiv.org/abs/1805.00020) is an extensive 65-page handbook going through the details.

## 0. Preparations

In [1]:
import logging

MadMiner uses the Python `logging` module to provide additional information and debugging output. You can choose how much of this output you want to see by switching the level in the following lines to `logging.DEBUG` or `logging.WARNING`.

In [2]:
# MadMiner output
logging.basicConfig(
    format="%(asctime)-5.5s %(name)-20.20s %(levelname)-7.7s %(message)s",
    datefmt="%H:%M",
    level=logging.INFO,
)

# Output of all other modules (e.g. matplotlib)
for key in logging.Logger.manager.loggerDict:
    if "madminer" not in key:
        logging.getLogger(key).setLevel(logging.WARNING)

In [3]:
from madminer import MadMiner, plot_2d_morphing_basis, plot_nd_morphing_basis_scatter, plot_nd_morphing_basis_slices

## 1. Input data

As an example process, we'll simulate VBF Higgs production with a decay into two photons. We'll try to measure two EFT coefficients that affect this process.

Have a look at the `cards` folder. You'll find text files ("cards") that specify the process simulation in typical MadGraph convention.

## 2. Define parameter space

After creating a `MadMiner` instance, the first important step is the definition of the parameter space. Each model parameter is characterized by a name as well as the LHA block and ID.

If morphing is used, one also has to specify the maximal power with which the parameter contributes to the squared matrix element. For instance, a parameter that contributes only to one vertex, will typically have `morphing_max_power=2`, while a parameter that contributes to two vertices usually has `morphing_max_power=4`. Exceptions arise for instance when the interference effects between the SM and dimension-six operators are modelled, but the square of the dimension-six amplitude (subleading in 1/Lambda) is not taken into account, in which case `morphing_max_power=1`. The `parameter_range` argument defines the range of parameter values that are used for the automatic optimization of the morphing basis.

Finally, the parameter values theta used internally by MadMiner and the parameter values written to the param_card (or reweight_card) given to MadGraph do not have to be exactly the same. With the option `parm_card_transform`, the user can supply a one-parameter function that maps a parameter value theta to the value given to MadGraph. This string is a python expression, in which `theta` is parsed as the parameter value. For instance, if the internal parameters are in the range (0, 1), but should be linearly scaled to (0, 100) in the param_card, one would have to use `param_card_transform="100*theta"`.

In [4]:
miner = MadMiner()
miner.add_parameter(
    lha_block="DIM6",
    lha_id=16,
    parameter_name="ctG",
    morphing_max_power=2,
    param_card_transform="1*theta",
    parameter_range=(-2.0, 2.0),
)
miner.add_parameter(
    lha_block="DIM6",
    lha_id=17,
    parameter_name="ctGI",
    morphing_max_power=2,
    param_card_transform="1*theta",
    parameter_range=(-2.0, 2.0),
)
miner.add_parameter(
    lha_block="DIM6",
    lha_id=51,
    parameter_name="cQq83",
    morphing_max_power=2,
    param_card_transform="1*theta",
    parameter_range=(-5.0, 5.0),
)
miner.add_parameter(
    lha_block="DIM6",
    lha_id=52,
    parameter_name="cQq81",
    morphing_max_power=2,
    param_card_transform="1*theta",
    parameter_range=(-5.0, 5.0),
)
miner.add_parameter(
    lha_block="DIM6",
    lha_id=53,
    parameter_name="cQu8",
    morphing_max_power=2,
    param_card_transform="1*theta",
    parameter_range=(-5.0, 5.0),
)
miner.add_parameter(
    lha_block="DIM6",
    lha_id=54,
    parameter_name="cQd8",
    morphing_max_power=2,
    param_card_transform="1*theta",
    parameter_range=(-5.0, 5.0),
)
miner.add_parameter(
    lha_block="DIM6",
    lha_id=55,
    parameter_name="ctq8",
    morphing_max_power=2,
    param_card_transform="1*theta",
    parameter_range=(-5.0, 5.0),
)
miner.add_parameter(
    lha_block="DIM6",
    lha_id=56,
    parameter_name="ctu8",
    morphing_max_power=2,
    param_card_transform="1*theta",
    parameter_range=(-5.0, 5.0),
)
miner.add_parameter(
    lha_block="DIM6",
    lha_id=57,
    parameter_name="ctd8",
    morphing_max_power=2,
    param_card_transform="1*theta",
    parameter_range=(-5.0, 5.0),
)

16:48 madminer.core.madmin INFO    Adding parameter: ctG (LHA: DIM6 16, Power: 2, Range: (-2.0, 2.0))
16:48 madminer.core.madmin WARNING Resetting benchmarks and morphing
16:48 madminer.core.madmin INFO    Adding parameter: ctGI (LHA: DIM6 17, Power: 2, Range: (-2.0, 2.0))
16:48 madminer.core.madmin WARNING Resetting benchmarks and morphing
16:48 madminer.core.madmin INFO    Adding parameter: cQq83 (LHA: DIM6 51, Power: 2, Range: (-5.0, 5.0))
16:48 madminer.core.madmin WARNING Resetting benchmarks and morphing
16:48 madminer.core.madmin INFO    Adding parameter: cQq81 (LHA: DIM6 52, Power: 2, Range: (-5.0, 5.0))
16:48 madminer.core.madmin WARNING Resetting benchmarks and morphing
16:48 madminer.core.madmin INFO    Adding parameter: cQu8 (LHA: DIM6 53, Power: 2, Range: (-5.0, 5.0))
16:48 madminer.core.madmin WARNING Resetting benchmarks and morphing
16:48 madminer.core.madmin INFO    Adding parameter: cQd8 (LHA: DIM6 54, Power: 2, Range: (-5.0, 5.0))
16:48 madminer.core.madmin WARNING R

## 3. Define benchmarks manually

The next step is the definition of all the points at which the weights (squared matrix elements) should be evaluated by MadGraph. We call these points "benchmarks". One can define benchmarks by hand:

In [5]:
miner.add_benchmark({"ctG": 0.0,"ctGI": 0.0, "cQq83": 0.0,"cQq81": 0.0,"cQu8": 0.0,"cQd8": 0.0,"ctq8": 0.0,"ctu8": 0.0,"ctd8": 0.0}, "sm")
miner.add_benchmark({"ctG": 2.0,"ctGI": 0.0, "cQq83": 0.0,"cQq81": 0.0,"cQu8": 0.0,"cQd8": 0.0,"ctq8": 0.0,"ctu8": 0.0,"ctd8": 0.0}, "ctG")
miner.add_benchmark({"ctG": -2.0,"ctGI": 0.0, "cQq83": 0.0,"cQq81": 0.0,"cQu8": 0.0,"cQd8": 0.0,"ctq8": 0.0,"ctu8": 0.0,"ctd8": 0.0},"neg_ctG")
miner.add_benchmark({"ctG": 0.0,"ctGI": 2.0, "cQq83": 0.0,"cQq81": 0.0,"cQu8": 0.0,"cQd8": 0.0,"ctq8": 0.0,"ctu8": 0.0,"ctd8": 0.0}, "ctGI")
miner.add_benchmark({"ctG": 0.0,"ctGI": -2.0, "cQq83": 0.0,"cQq81": 0.0,"cQu8": 0.0,"cQd8": 0.0,"ctq8": 0.0,"ctu8": 0.0,"ctd8": 0.0},"neg_ctGI")
miner.add_benchmark({"ctG": 0.0,"ctGI": 0.0, "cQq83": 5.0,"cQq81": 0.0,"cQu8": 0.0,"cQd8": 0.0,"ctq8": 0.0,"ctu8": 0.0,"ctd8": 0.0}, "cQq83")
miner.add_benchmark({"ctG": 0.0,"ctGI": 0.0, "cQq83": -5.0,"cQq81": 0.0,"cQu8": 0.0,"cQd8": 0.0,"ctq8": 0.0,"ctu8": 0.0,"ctd8": 0.0}, "neg_cQq83")
miner.add_benchmark({"ctG": 0.0,"ctGI": 0.0, "cQq83": 0.0,"cQq81": 5.0,"cQu8": 0.0,"cQd8": 0.0,"ctq8": 0.0,"ctu8": 0.0,"ctd8": 0.0}, "cQq81")
miner.add_benchmark({"ctG": 0.0,"ctGI": 0.0, "cQq83": 0.0,"cQq81": -5.0,"cQu8": 0.0,"cQd8": 0.0,"ctq8": 0.0,"ctu8": 0.0,"ctd8": 0.0}, "neg_cQq81")
miner.add_benchmark({"ctG": 0.0,"ctGI": 0.0, "cQq83": 0.0,"cQq81": 0.0,"cQu8": 5.0,"cQd8": 0.0,"ctq8": 0.0,"ctu8": 0.0,"ctd8": 0.0}, "cQu8")
miner.add_benchmark({"ctG": 0.0,"ctGI": 0.0, "cQq83": 0.0,"cQq81": 0.0,"cQu8": -5.0,"cQd8": 0.0,"ctq8": 0.0,"ctu8": 0.0,"ctd8": 0.0}, "neg_cQu8")
miner.add_benchmark({"ctG": 0.0,"ctGI": 0.0, "cQq83": 0.0,"cQq81": 0.0,"cQu8": 0.0,"cQd8": 5.0,"ctq8": 0.0,"ctu8": 0.0,"ctd8": 0.0}, "cQd8")
miner.add_benchmark({"ctG": 0.0,"ctGI": 0.0, "cQq83": 0.0,"cQq81": 0.0,"cQu8": 0.0,"cQd8": -5.0,"ctq8": 0.0,"ctu8": 0.0,"ctd8": 0.0}, "neg_cQd8")
miner.add_benchmark({"ctG": 0.0,"ctGI": 0.0, "cQq83": 0.0,"cQq81": 0.0,"cQu8": 0.0,"cQd8": 0.0,"ctq8": 5.0,"ctu8": 0.0,"ctd8": 0.0}, "ctq8")
miner.add_benchmark({"ctG": 0.0,"ctGI": 0.0, "cQq83": 0.0,"cQq81": 0.0,"cQu8": 0.0,"cQd8": 0.0,"ctq8": -5.0,"ctu8": 0.0,"ctd8": 0.0},"neg_ctq8")
miner.add_benchmark({"ctG": 0.0,"ctGI": 0.0, "cQq83": 0.0,"cQq81": 0.0,"cQu8": 0.0,"cQd8": 0.0,"ctq8": 0.0,"ctu8": 5.0,"ctd8": 0.0}, "ctu8")
miner.add_benchmark({"ctG": 0.0,"ctGI": 0.0, "cQq83": 0.0,"cQq81": 0.0,"cQu8": 0.0,"cQd8": 0.0,"ctq8": 0.0,"ctu8": -5.0,"ctd8": 0.0}, "neg_ctu8")
miner.add_benchmark({"ctG": 0.0,"ctGI": 0.0, "cQq83": 0.0,"cQq81": 0.0,"cQu8": 0.0,"cQd8": 0.0,"ctq8": 0.0,"ctu8": 0.0,"ctd8": 5.0}, "ctd8")
miner.add_benchmark({"ctG": 0.0,"ctGI": 0.0, "cQq83": 0.0,"cQq81": 0.0,"cQu8": 0.0,"cQd8": 0.0,"ctq8": 0.0,"ctu8": 0.0,"ctd8": -5.0}, "neg_ctd8")


16:48 madminer.core.madmin INFO    Added benchmark sm: ctG = 0.00e+00, ctGI = 0.00e+00, cQq83 = 0.00e+00, cQq81 = 0.00e+00, cQu8 = 0.00e+00, cQd8 = 0.00e+00, ctq8 = 0.00e+00, ctu8 = 0.00e+00, ctd8 = 0.00e+00
16:48 madminer.core.madmin INFO    Added benchmark ctG: ctG = 2.00, ctGI = 0.00e+00, cQq83 = 0.00e+00, cQq81 = 0.00e+00, cQu8 = 0.00e+00, cQd8 = 0.00e+00, ctq8 = 0.00e+00, ctu8 = 0.00e+00, ctd8 = 0.00e+00
16:48 madminer.core.madmin INFO    Added benchmark neg_ctG: ctG = -2.00e+00, ctGI = 0.00e+00, cQq83 = 0.00e+00, cQq81 = 0.00e+00, cQu8 = 0.00e+00, cQd8 = 0.00e+00, ctq8 = 0.00e+00, ctu8 = 0.00e+00, ctd8 = 0.00e+00
16:48 madminer.core.madmin INFO    Added benchmark ctGI: ctG = 0.00e+00, ctGI = 2.00, cQq83 = 0.00e+00, cQq81 = 0.00e+00, cQu8 = 0.00e+00, cQd8 = 0.00e+00, ctq8 = 0.00e+00, ctu8 = 0.00e+00, ctd8 = 0.00e+00
16:48 madminer.core.madmin INFO    Added benchmark neg_ctGI: ctG = 0.00e+00, ctGI = -2.00e+00, cQq83 = 0.00e+00, cQq81 = 0.00e+00, cQu8 = 0.00e+00, cQd8 = 0.00e+00, ct

In [6]:
miner.add_benchmark({"ctG": 2.0,"ctGI": 2.0, "cQq83": 0.0,"cQq81": 0.0,"cQu8": 0.0,"cQd8": 0.0,"ctq8": 0.0,"ctu8": 0.0,"ctd8": 0.0}, "ctG_ctGI")
miner.add_benchmark({"ctG": 2.0,"ctGI": 0.0, "cQq83": 5.0,"cQq81": 0.0,"cQu8": 0.0,"cQd8": 0.0,"ctq8": 0.0,"ctu8": 0.0,"ctd8": 0.0}, "ctG_cQq83")
miner.add_benchmark({"ctG": 2.0,"ctGI": 0.0, "cQq83": 0.0,"cQq81": 5.0,"cQu8": 0.0,"cQd8": 0.0,"ctq8": 0.0,"ctu8": 0.0,"ctd8": 0.0}, "ctG_cQq81")
miner.add_benchmark({"ctG": 2.0,"ctGI": 0.0, "cQq83": 0.0,"cQq81": 0.0,"cQu8": 5.0,"cQd8": 0.0,"ctq8": 0.0,"ctu8": 0.0,"ctd8": 0.0}, "ctG_cQu8")
miner.add_benchmark({"ctG": 2.0,"ctGI": 0.0, "cQq83": 0.0,"cQq81": 0.0,"cQu8": 0.0,"cQd8": 5.0,"ctq8": 0.0,"ctu8": 0.0,"ctd8": 0.0}, "ctG_cQd8")
miner.add_benchmark({"ctG": 2.0,"ctGI": 0.0, "cQq83": 0.0,"cQq81": 0.0,"cQu8": 0.0,"cQd8": 0.0,"ctq8": 5.0,"ctu8": 0.0,"ctd8": 0.0}, "ctG_ctq8")
miner.add_benchmark({"ctG": 2.0,"ctGI": 0.0, "cQq83": 0.0,"cQq81": 0.0,"cQu8": 0.0,"cQd8": 0.0,"ctq8": 0.0,"ctu8": 5.0,"ctd8": 0.0}, "ctG_ctu8")
miner.add_benchmark({"ctG": 2.0,"ctGI": 0.0, "cQq83": 0.0,"cQq81": 0.0,"cQu8": 0.0,"cQd8": 0.0,"ctq8": 0.0,"ctu8": 0.0,"ctd8": 5.0}, "ctG_ctd8")

16:49 madminer.core.madmin INFO    Added benchmark ctG_ctGI: ctG = 2.00, ctGI = 2.00, cQq83 = 0.00e+00, cQq81 = 0.00e+00, cQu8 = 0.00e+00, cQd8 = 0.00e+00, ctq8 = 0.00e+00, ctu8 = 0.00e+00, ctd8 = 0.00e+00
16:49 madminer.core.madmin INFO    Added benchmark ctG_cQq83: ctG = 2.00, ctGI = 0.00e+00, cQq83 = 5.00, cQq81 = 0.00e+00, cQu8 = 0.00e+00, cQd8 = 0.00e+00, ctq8 = 0.00e+00, ctu8 = 0.00e+00, ctd8 = 0.00e+00
16:49 madminer.core.madmin INFO    Added benchmark ctG_cQq81: ctG = 2.00, ctGI = 0.00e+00, cQq83 = 0.00e+00, cQq81 = 5.00, cQu8 = 0.00e+00, cQd8 = 0.00e+00, ctq8 = 0.00e+00, ctu8 = 0.00e+00, ctd8 = 0.00e+00
16:49 madminer.core.madmin INFO    Added benchmark ctG_cQu8: ctG = 2.00, ctGI = 0.00e+00, cQq83 = 0.00e+00, cQq81 = 0.00e+00, cQu8 = 5.00, cQd8 = 0.00e+00, ctq8 = 0.00e+00, ctu8 = 0.00e+00, ctd8 = 0.00e+00
16:49 madminer.core.madmin INFO    Added benchmark ctG_cQd8: ctG = 2.00, ctGI = 0.00e+00, cQq83 = 0.00e+00, cQq81 = 0.00e+00, cQu8 = 0.00e+00, cQd8 = 5.00, ctq8 = 0.00e+00, c

In [7]:
miner.add_benchmark({"ctG":0.0 ,"ctGI": 2.0, "cQq83": 5.0,"cQq81": 0.0,"cQu8": 0.0,"cQd8": 0.0,"ctq8": 0.0,"ctu8": 0.0,"ctd8": 0.0}, "ctGI_cQq83")
miner.add_benchmark({"ctG":0.0 ,"ctGI": 2.0, "cQq83": 0.0,"cQq81": 5.0,"cQu8": 0.0,"cQd8": 0.0,"ctq8": 0.0,"ctu8": 0.0,"ctd8": 0.0}, "ctGI_cQq81")
miner.add_benchmark({"ctG":0.0 ,"ctGI": 2.0, "cQq83": 0.0,"cQq81": 0.0,"cQu8": 5.0,"cQd8": 0.0,"ctq8": 0.0,"ctu8": 0.0,"ctd8": 0.0}, "ctGI_cQu8")
miner.add_benchmark({"ctG":0.0 ,"ctGI": 2.0, "cQq83": 0.0,"cQq81": 0.0,"cQu8": 0.0,"cQd8": 5.0,"ctq8": 0.0,"ctu8": 0.0,"ctd8": 0.0}, "ctGI_cQd8")
miner.add_benchmark({"ctG":0.0 ,"ctGI": 2.0, "cQq83": 0.0,"cQq81": 0.0,"cQu8": 0.0,"cQd8": 0.0,"ctq8": 5.0,"ctu8": 0.0,"ctd8": 0.0}, "ctGI_ctq8")
miner.add_benchmark({"ctG":0.0 ,"ctGI": 2.0, "cQq83": 0.0,"cQq81": 0.0,"cQu8": 0.0,"cQd8": 0.0,"ctq8": 0.0,"ctu8": 5.0,"ctd8": 0.0}, "ctGI_ctu8")
miner.add_benchmark({"ctG":0.0 ,"ctGI": 2.0, "cQq83": 0.0,"cQq81": 0.0,"cQu8": 0.0,"cQd8": 0.0,"ctq8": 0.0,"ctu8": 0.0,"ctd8": 5.0}, "ctGI_ctd8")

16:50 madminer.core.madmin INFO    Added benchmark ctGI_cQq83: ctG = 0.00e+00, ctGI = 2.00, cQq83 = 5.00, cQq81 = 0.00e+00, cQu8 = 0.00e+00, cQd8 = 0.00e+00, ctq8 = 0.00e+00, ctu8 = 0.00e+00, ctd8 = 0.00e+00
16:50 madminer.core.madmin INFO    Added benchmark ctGI_cQq81: ctG = 0.00e+00, ctGI = 2.00, cQq83 = 0.00e+00, cQq81 = 5.00, cQu8 = 0.00e+00, cQd8 = 0.00e+00, ctq8 = 0.00e+00, ctu8 = 0.00e+00, ctd8 = 0.00e+00
16:50 madminer.core.madmin INFO    Added benchmark ctGI_cQu8: ctG = 0.00e+00, ctGI = 2.00, cQq83 = 0.00e+00, cQq81 = 0.00e+00, cQu8 = 5.00, cQd8 = 0.00e+00, ctq8 = 0.00e+00, ctu8 = 0.00e+00, ctd8 = 0.00e+00
16:50 madminer.core.madmin INFO    Added benchmark ctGI_cQd8: ctG = 0.00e+00, ctGI = 2.00, cQq83 = 0.00e+00, cQq81 = 0.00e+00, cQu8 = 0.00e+00, cQd8 = 5.00, ctq8 = 0.00e+00, ctu8 = 0.00e+00, ctd8 = 0.00e+00
16:50 madminer.core.madmin INFO    Added benchmark ctGI_ctq8: ctG = 0.00e+00, ctGI = 2.00, cQq83 = 0.00e+00, cQq81 = 0.00e+00, cQu8 = 0.00e+00, cQd8 = 0.00e+00, ctq8 = 5.

In [8]:
miner.add_benchmark({"ctG":0.0 ,"ctGI": 0.0, "cQq83": 5.0,"cQq81": 5.0,"cQu8": 0.0,"cQd8": 0.0,"ctq8": 0.0,"ctu8": 0.0,"ctd8": 0.0}, "cQq83_cQq81")
miner.add_benchmark({"ctG":0.0 ,"ctGI": 0.0, "cQq83":5.0,"cQq81": 0.0,"cQu8": 5.0,"cQd8": 0.0,"ctq8": 0.0,"ctu8": 0.0,"ctd8": 0.0}, "cQq83_cQu8")
miner.add_benchmark({"ctG":0.0 ,"ctGI": 0.0, "cQq83": 5.0,"cQq81": 0.0,"cQu8": 0.0,"cQd8": 5.0,"ctq8": 0.0,"ctu8": 0.0,"ctd8": 0.0}, "cQq83_cQd8")
miner.add_benchmark({"ctG":0.0 ,"ctGI": 0.0, "cQq83": 5.0,"cQq81": 0.0,"cQu8": 0.0,"cQd8": 0.0,"ctq8": 5.0,"ctu8": 0.0,"ctd8": 0.0}, "cQq83_ctq8")
miner.add_benchmark({"ctG":0.0 ,"ctGI": 0.0, "cQq83": 5.0,"cQq81": 0.0,"cQu8": 0.0,"cQd8": 0.0,"ctq8": 0.0,"ctu8": 5.0,"ctd8": 0.0}, "cQq83_ctu8")
miner.add_benchmark({"ctG":0.0 ,"ctGI": 0.0, "cQq83": 5.0,"cQq81": 0.0,"cQu8": 0.0,"cQd8": 0.0,"ctq8": 0.0,"ctu8": 0.0,"ctd8": 5.0}, "cQq83_ctd8")

16:51 madminer.core.madmin INFO    Added benchmark cQq83_cQq81: ctG = 0.00e+00, ctGI = 0.00e+00, cQq83 = 5.00, cQq81 = 5.00, cQu8 = 0.00e+00, cQd8 = 0.00e+00, ctq8 = 0.00e+00, ctu8 = 0.00e+00, ctd8 = 0.00e+00
16:51 madminer.core.madmin INFO    Added benchmark cQq83_cQu8: ctG = 0.00e+00, ctGI = 0.00e+00, cQq83 = 5.00, cQq81 = 0.00e+00, cQu8 = 5.00, cQd8 = 0.00e+00, ctq8 = 0.00e+00, ctu8 = 0.00e+00, ctd8 = 0.00e+00
16:51 madminer.core.madmin INFO    Added benchmark cQq83_cQd8: ctG = 0.00e+00, ctGI = 0.00e+00, cQq83 = 5.00, cQq81 = 0.00e+00, cQu8 = 0.00e+00, cQd8 = 5.00, ctq8 = 0.00e+00, ctu8 = 0.00e+00, ctd8 = 0.00e+00
16:51 madminer.core.madmin INFO    Added benchmark cQq83_ctq8: ctG = 0.00e+00, ctGI = 0.00e+00, cQq83 = 5.00, cQq81 = 0.00e+00, cQu8 = 0.00e+00, cQd8 = 0.00e+00, ctq8 = 5.00, ctu8 = 0.00e+00, ctd8 = 0.00e+00
16:51 madminer.core.madmin INFO    Added benchmark cQq83_ctu8: ctG = 0.00e+00, ctGI = 0.00e+00, cQq83 = 5.00, cQq81 = 0.00e+00, cQu8 = 0.00e+00, cQd8 = 0.00e+00, ctq8 

In [9]:
miner.add_benchmark({"ctG":0.0 ,"ctGI": 0.0, "cQq83":0.0,"cQq81": 5.0,"cQu8": 5.0,"cQd8": 0.0,"ctq8": 0.0,"ctu8": 0.0,"ctd8": 0.0}, "cQq81_cQu8")
miner.add_benchmark({"ctG":0.0 ,"ctGI": 0.0, "cQq83": 0.0,"cQq81": 5.0,"cQu8": 0.0,"cQd8": 5.0,"ctq8": 0.0,"ctu8": 0.0,"ctd8": 0.0}, "cQq81_cQd8")
miner.add_benchmark({"ctG":0.0 ,"ctGI": 0.0, "cQq83": 0.0,"cQq81": 5.0,"cQu8": 0.0,"cQd8": 0.0,"ctq8": 5.0,"ctu8": 0.0,"ctd8": 0.0}, "cQq81_ctq8")
miner.add_benchmark({"ctG":0.0 ,"ctGI": 0.0, "cQq83": 0.0,"cQq81": 5.0,"cQu8": 0.0,"cQd8": 0.0,"ctq8": 0.0,"ctu8": 5.0,"ctd8": 0.0}, "cQq81_ctu8")
miner.add_benchmark({"ctG":0.0 ,"ctGI": 0.0, "cQq83": 0.0,"cQq81": 5.0,"cQu8": 0.0,"cQd8": 0.0,"ctq8": 0.0,"ctu8": 0.0,"ctd8": 5.0}, "cQq81_ctd8")

16:51 madminer.core.madmin INFO    Added benchmark cQq81_cQu8: ctG = 0.00e+00, ctGI = 0.00e+00, cQq83 = 0.00e+00, cQq81 = 5.00, cQu8 = 5.00, cQd8 = 0.00e+00, ctq8 = 0.00e+00, ctu8 = 0.00e+00, ctd8 = 0.00e+00
16:51 madminer.core.madmin INFO    Added benchmark cQq81_cQd8: ctG = 0.00e+00, ctGI = 0.00e+00, cQq83 = 0.00e+00, cQq81 = 5.00, cQu8 = 0.00e+00, cQd8 = 5.00, ctq8 = 0.00e+00, ctu8 = 0.00e+00, ctd8 = 0.00e+00
16:51 madminer.core.madmin INFO    Added benchmark cQq81_ctq8: ctG = 0.00e+00, ctGI = 0.00e+00, cQq83 = 0.00e+00, cQq81 = 5.00, cQu8 = 0.00e+00, cQd8 = 0.00e+00, ctq8 = 5.00, ctu8 = 0.00e+00, ctd8 = 0.00e+00
16:51 madminer.core.madmin INFO    Added benchmark cQq81_ctu8: ctG = 0.00e+00, ctGI = 0.00e+00, cQq83 = 0.00e+00, cQq81 = 5.00, cQu8 = 0.00e+00, cQd8 = 0.00e+00, ctq8 = 0.00e+00, ctu8 = 5.00, ctd8 = 0.00e+00
16:51 madminer.core.madmin INFO    Added benchmark cQq81_ctd8: ctG = 0.00e+00, ctGI = 0.00e+00, cQq83 = 0.00e+00, cQq81 = 5.00, cQu8 = 0.00e+00, cQd8 = 0.00e+00, ctq8 =

In [10]:
miner.add_benchmark({"ctG":0.0 ,"ctGI": 0.0, "cQq83": 0.0,"cQq81": 0.0,"cQu8": 5.0,"cQd8": 5.0,"ctq8": 0.0,"ctu8": 0.0,"ctd8": 0.0}, "cQu8_cQd8")
miner.add_benchmark({"ctG":0.0 ,"ctGI": 0.0, "cQq83": 0.0,"cQq81": 0.0,"cQu8": 5.0,"cQd8": 0.0,"ctq8": 5.0,"ctu8": 0.0,"ctd8": 0.0}, "cQu8_ctq8")
miner.add_benchmark({"ctG":0.0 ,"ctGI": 0.0, "cQq83": 0.0,"cQq81": 0.0,"cQu8": 5.0,"cQd8": 0.0,"ctq8": 0.0,"ctu8": 5.0,"ctd8": 0.0}, "cQu8_ctu8")
miner.add_benchmark({"ctG":0.0 ,"ctGI": 0.0, "cQq83": 0.0,"cQq81": 0.0,"cQu8": 5.0,"cQd8": 0.0,"ctq8": 0.0,"ctu8": 0.0,"ctd8": 5.0}, "cQu8_ctd8")

16:52 madminer.core.madmin INFO    Added benchmark cQu8_cQd8: ctG = 0.00e+00, ctGI = 0.00e+00, cQq83 = 0.00e+00, cQq81 = 0.00e+00, cQu8 = 5.00, cQd8 = 5.00, ctq8 = 0.00e+00, ctu8 = 0.00e+00, ctd8 = 0.00e+00
16:52 madminer.core.madmin INFO    Added benchmark cQu8_ctq8: ctG = 0.00e+00, ctGI = 0.00e+00, cQq83 = 0.00e+00, cQq81 = 0.00e+00, cQu8 = 5.00, cQd8 = 0.00e+00, ctq8 = 5.00, ctu8 = 0.00e+00, ctd8 = 0.00e+00
16:52 madminer.core.madmin INFO    Added benchmark cQu8_ctu8: ctG = 0.00e+00, ctGI = 0.00e+00, cQq83 = 0.00e+00, cQq81 = 0.00e+00, cQu8 = 5.00, cQd8 = 0.00e+00, ctq8 = 0.00e+00, ctu8 = 5.00, ctd8 = 0.00e+00
16:52 madminer.core.madmin INFO    Added benchmark cQu8_ctd8: ctG = 0.00e+00, ctGI = 0.00e+00, cQq83 = 0.00e+00, cQq81 = 0.00e+00, cQu8 = 5.00, cQd8 = 0.00e+00, ctq8 = 0.00e+00, ctu8 = 0.00e+00, ctd8 = 5.00


In [11]:
miner.add_benchmark({"ctG":0.0 ,"ctGI": 0.0, "cQq83": 0.0,"cQq81": 0.0,"cQu8": 0.0,"cQd8": 5.0,"ctq8": 5.0,"ctu8": 0.0,"ctd8": 0.0}, "cQd8_ctq8")
miner.add_benchmark({"ctG":0.0 ,"ctGI": 0.0, "cQq83": 0.0,"cQq81": 0.0,"cQu8": 0.0,"cQd8": 5.0,"ctq8": 0.0,"ctu8": 5.0,"ctd8": 0.0}, "cQd8_ctu8")
miner.add_benchmark({"ctG":0.0 ,"ctGI": 0.0, "cQq83": 0.0,"cQq81": 0.0,"cQu8": 0.0,"cQd8": 5.0,"ctq8": 0.0,"ctu8": 0.0,"ctd8": 5.0}, "cQd8_ctd8")

16:52 madminer.core.madmin INFO    Added benchmark cQd8_ctq8: ctG = 0.00e+00, ctGI = 0.00e+00, cQq83 = 0.00e+00, cQq81 = 0.00e+00, cQu8 = 0.00e+00, cQd8 = 5.00, ctq8 = 5.00, ctu8 = 0.00e+00, ctd8 = 0.00e+00
16:52 madminer.core.madmin INFO    Added benchmark cQd8_ctu8: ctG = 0.00e+00, ctGI = 0.00e+00, cQq83 = 0.00e+00, cQq81 = 0.00e+00, cQu8 = 0.00e+00, cQd8 = 5.00, ctq8 = 0.00e+00, ctu8 = 5.00, ctd8 = 0.00e+00
16:52 madminer.core.madmin INFO    Added benchmark cQd8_ctd8: ctG = 0.00e+00, ctGI = 0.00e+00, cQq83 = 0.00e+00, cQq81 = 0.00e+00, cQu8 = 0.00e+00, cQd8 = 5.00, ctq8 = 0.00e+00, ctu8 = 0.00e+00, ctd8 = 5.00


In [12]:
miner.add_benchmark({"ctG":0.0 ,"ctGI": 0.0, "cQq83": 0.0,"cQq81": 0.0,"cQu8": 0.0,"cQd8": 0.0,"ctq8": 5.0,"ctu8": 5.0,"ctd8": 0.0}, "ctq8_ctu8")
miner.add_benchmark({"ctG":0.0 ,"ctGI": 0.0, "cQq83": 0.0,"cQq81": 0.0,"cQu8": 0.0,"cQd8": 0.0,"ctq8": 5.0,"ctu8": 0.0,"ctd8": 5.0}, "ctq8_ctd8")

16:52 madminer.core.madmin INFO    Added benchmark ctq8_ctu8: ctG = 0.00e+00, ctGI = 0.00e+00, cQq83 = 0.00e+00, cQq81 = 0.00e+00, cQu8 = 0.00e+00, cQd8 = 0.00e+00, ctq8 = 5.00, ctu8 = 5.00, ctd8 = 0.00e+00
16:52 madminer.core.madmin INFO    Added benchmark ctq8_ctd8: ctG = 0.00e+00, ctGI = 0.00e+00, cQq83 = 0.00e+00, cQq81 = 0.00e+00, cQu8 = 0.00e+00, cQd8 = 0.00e+00, ctq8 = 5.00, ctu8 = 0.00e+00, ctd8 = 5.00


In [13]:
miner.add_benchmark({"ctG":0.0 ,"ctGI": 0.0, "cQq83": 0.0,"cQq81": 0.0,"cQu8": 0.0,"cQd8": 0.0,"ctq8": 0.0,"ctu8": 5.0,"ctd8": 5.0}, "ctu8_ctd8")

16:52 madminer.core.madmin INFO    Added benchmark ctu8_ctd8: ctG = 0.00e+00, ctGI = 0.00e+00, cQq83 = 0.00e+00, cQq81 = 0.00e+00, cQu8 = 0.00e+00, cQd8 = 0.00e+00, ctq8 = 0.00e+00, ctu8 = 5.00, ctd8 = 5.00


## 4. Morphing

Morphing is a powerful technique that allows for the fast and exact interpolation of event weights (or any derived quantity) from a set of original benchmark points (the "morphing basis") to anywhere in theory parameter space. This procedure depends on the powers with which the parameters affect the differential cross sections (this is why `add_parameter()` wants to know about this). Please see Sec. II.C.2 of ["A Guide to Constraining Effective Field Theories With Machine Learning"](https://arxiv.org/abs/1805.00020) for an explanation of the technique.

Morphing is not strictly necessary to run MadMiner, but it makes life easier in so many ways (and many methods are so far only implemented for morphing-based problems). We strongly recommend to use this wherever possible.

To activate morphing, the function `set_morphing` has to be called. With the option `include_existing_benchmarks=True`, MadMiner will keep all the benchmark points defined beforehand and run a simple optimization algorithm to fix the remaining ones for the basis (which may be none). Otherwise, MadMiner will optimize the full basis and forget about all previously defined benchmark points.

The other important keyword is `max_overall_power`. This sets the maximal combined power with which all parameters together contribute to the squared matrix element (cross section). This constraint is in addition to the `morphing_max_power` keyword of `add_parameter()` (see above). For instance, if there are two parameters of interest `a` and `b`, and it is known that there are contributions proportional to `1` (constant), `a`, `a^2`, `b`, `b^2`, and `ab`, then `max_overall_power=2`. If there are also contributions proportional to `a^2 b`, `a b^2`, and `a^2 b^2`, then `max_overall_power=4`.

In [14]:
miner.set_morphing(include_existing_benchmarks=True, max_overall_power=2)

16:52 madminer.core.madmin INFO    Optimizing basis for morphing
16:53 madminer.core.madmin INFO    Set up morphing with 9 parameters, 55 morphing components, 55 predefined basis points, and 0 new basis points


Let's have a look at the resulting morphing basis and the "morphing error", i.e. the sum of squared morphing weights as a function of the parameter space. The black dots mark the benchmarks at which the squared matrix element will be evaluated when MadGraph is run. In between them, the morphing will interpolate. Since the morphing knows about the functional form of the amplitudes, there is only a small numerical uncertainty in this interpolation, we expect that the color in this plot is indicative of this uncertainty.

In [15]:
madminer.plotting.morphing?

Object `madminer.plotting.morphing` not found.


In [16]:
fig = plot_2d_morphing_basis(
    miner.morpher,
    xlabel=r"$c_{tG} / \Lambda^2$",
    ylabel=r"$c_{tq8} / \Lambda^2$",
    xrange=(-2.1, 2.1),
    yrange=(-5.1, 5.1),
)

AssertionError: Only 2d problems can be plotted with this function

Note that squared weights (the colormap here) up to 1000 or even 10000 can still be perfectly fine and are in fact sometimes unavoidable.

## 5. Save setup

The parameter space, benchmark points, and morphing setup are saved in a HDF5 file:

In [17]:
miner.save("data/setup-complete_ttbarLOset-test.h5")

16:53 madminer.core.madmin INFO    Saving setup (including morphing) to data/setup-complete_ttbarLOset-test.h5


They can now be loaded again with `miner.load(filename)`.

That's it for the setup (we'll only add one step when talking about systematic uncertainties in part 5 of the tutorial). Please continue with part 2a **or** part 2b, depending on whether you want to run the faster parton-level analysis or the more realistic Delphes-level analysis.

In [18]:
import h5py
import numpy as np
setupfile = "data/setup-complete_ttbarLOset-test.h5"
f = h5py.File(setupfile)
f.keys()


<KeysViewHDF5 ['benchmarks', 'morphing', 'parameters']>

In [19]:
for key in f.keys():
    print("--------------------------------------------------------------")
    print(key+":")
    for string in np.array(f[key]):
        print("  ",string +":", np.array(f[key][string]))
    print
f.close()

--------------------------------------------------------------
benchmarks:
   is_nuisance: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
   is_reference: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
   names: [b'sm' b'ctG' b'neg_ctG' b'ctGI' b'neg_ctGI' b'cQq83' b'neg_cQq83'
 b'cQq81' b'neg_cQq81' b'cQu8' b'neg_cQu8' b'cQd8' b'neg_cQd8' b'ctq8'
 b'neg_ctq8' b'ctu8' b'neg_ctu8' b'ctd8' b'neg_ctd8' b'ctG_ctGI'
 b'ctG_cQq83' b'ctG_cQq81' b'ctG_cQu8' b'ctG_cQd8' b'ctG_ctq8' b'ctG_ctu8'
 b'ctG_ctd8' b'ctGI_cQq83' b'ctGI_cQq81' b'ctGI_cQu8' b'ctGI_cQd8'
 b'ctGI_ctq8' b'ctGI_ctu8' b'ctGI_ctd8' b'cQq83_cQq81' b'cQq83_cQu8'
 b'cQq83_cQd8' b'cQq83_ctq8' b'cQq83_ctu8' b'cQq83_ctd8' b'cQq81_cQu8'
 b'cQq81_cQd8' b'cQq81_ctq8' b'cQq81_ctu8' b'cQq81_ctd8' b'cQu8_cQd8'
 b'cQu8_ctq8' b'cQu8_ctu8' b'cQu8_ctd8' b'cQd8_ctq8' b'cQd8_ctu8'
 b'cQd8_ctd8' b'ctq8_ctu8' b'ctq8_c

In [20]:
import h5py
import numpy as np
setupfile = "data/setup-complete_ttbarLOset-test.h5"
f = h5py.File(setupfile)
print(np.array(f['benchmarks']['values']))



[[ 0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 2.  0.  0.  0.  0.  0.  0.  0.  0.]
 [-2.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  2.  0.  0.  0.  0.  0.  0.  0.]
 [ 0. -2.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  5.  0.  0.  0.  0.  0.  0.]
 [ 0.  0. -5.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  5.  0.  0.  0.  0.  0.]
 [ 0.  0.  0. -5.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  5.  0.  0.  0.  0.]
 [ 0.  0.  0.  0. -5.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  5.  0.  0.  0.]
 [ 0.  0.  0.  0.  0. -5.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  5.  0.  0.]
 [ 0.  0.  0.  0.  0.  0. -5.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  5.  0.]
 [ 0.  0.  0.  0.  0.  0.  0. -5.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  5.]
 [ 0.  0.  0.  0.  0.  0.  0.  0. -5.]
 [ 2.  2.  0.  0.  0.  0.  0.  0.  0.]
 [ 2.  0.  5.  0.  0.  0.  0.  0.  0.]
 [ 2.  0.  0.  5.  0.  0.  0.  0.  0.]
 [ 2.  0.  0.  0.  5.  0.  0.  0.  0.]
 [ 2.  0.  0.  0.  0.  5.  0.  0.  0.]
 [ 2.  0.  0.  0.  0.  0.  5.  0.  0.]
 [ 2.  0.  0.  0.  0.  0.

In [21]:
len(np.array(f['benchmarks']['values']))

55

In [22]:
nscenarios=len(np.array(f['benchmarks']['values']))
ncouplings=len(np.array(f['benchmarks']['values'][0]))
i=0
dim6topMatrix=np.zeros((nscenarios,ncouplings))
for scenario in np.array(f['benchmarks']['values']):
    j=0
    for coupling in scenario:
        dim6topMatrix[i,j] = coupling
        j=j+1
    i=i+1

In [23]:
print(dim6topMatrix)

[[ 0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 2.  0.  0.  0.  0.  0.  0.  0.  0.]
 [-2.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  2.  0.  0.  0.  0.  0.  0.  0.]
 [ 0. -2.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  5.  0.  0.  0.  0.  0.  0.]
 [ 0.  0. -5.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  5.  0.  0.  0.  0.  0.]
 [ 0.  0.  0. -5.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  5.  0.  0.  0.  0.]
 [ 0.  0.  0.  0. -5.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  5.  0.  0.  0.]
 [ 0.  0.  0.  0.  0. -5.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  5.  0.  0.]
 [ 0.  0.  0.  0.  0.  0. -5.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  5.  0.]
 [ 0.  0.  0.  0.  0.  0.  0. -5.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  5.]
 [ 0.  0.  0.  0.  0.  0.  0.  0. -5.]
 [ 2.  2.  0.  0.  0.  0.  0.  0.  0.]
 [ 2.  0.  5.  0.  0.  0.  0.  0.  0.]
 [ 2.  0.  0.  5.  0.  0.  0.  0.  0.]
 [ 2.  0.  0.  0.  5.  0.  0.  0.  0.]
 [ 2.  0.  0.  0.  0.  5.  0.  0.  0.]
 [ 2.  0.  0.  0.  0.  0.  5.  0.  0.]
 [ 2.  0.  0.  0.  0.  0.

In [24]:
nscenarios=len(np.array(f['benchmarks']['values']))
dim6top = [ "ctG","ctGI", "cQq83","cQq81","cQu8","cQd8","ctq8","ctu8","ctd8" ]
for i in range(len(dim6top)):
    COUPTAG = ("COUPSDIM6TOP[%s]=' %s ;"%(i,dim6top[i]))
    j=0
    for j in range(nscenarios):
        COUPTAG = COUPTAG + (" %.4g ; "%dim6topMatrix[j,i]) 
    COUPTAG = COUPTAG + (" %.4g ' "%dim6topMatrix[len(dim6top),i]) 
    print(COUPTAG)

COUPSDIM6TOP[0]=' ctG ; 0 ;  2 ;  -2 ;  0 ;  0 ;  0 ;  0 ;  0 ;  0 ;  0 ;  0 ;  0 ;  0 ;  0 ;  0 ;  0 ;  0 ;  0 ;  0 ;  2 ;  2 ;  2 ;  2 ;  2 ;  2 ;  2 ;  2 ;  0 ;  0 ;  0 ;  0 ;  0 ;  0 ;  0 ;  0 ;  0 ;  0 ;  0 ;  0 ;  0 ;  0 ;  0 ;  0 ;  0 ;  0 ;  0 ;  0 ;  0 ;  0 ;  0 ;  0 ;  0 ;  0 ;  0 ;  0 ;  0 ' 
COUPSDIM6TOP[1]=' ctGI ; 0 ;  0 ;  0 ;  2 ;  -2 ;  0 ;  0 ;  0 ;  0 ;  0 ;  0 ;  0 ;  0 ;  0 ;  0 ;  0 ;  0 ;  0 ;  0 ;  2 ;  0 ;  0 ;  0 ;  0 ;  0 ;  0 ;  0 ;  2 ;  2 ;  2 ;  2 ;  2 ;  2 ;  2 ;  0 ;  0 ;  0 ;  0 ;  0 ;  0 ;  0 ;  0 ;  0 ;  0 ;  0 ;  0 ;  0 ;  0 ;  0 ;  0 ;  0 ;  0 ;  0 ;  0 ;  0 ;  0 ' 
COUPSDIM6TOP[2]=' cQq83 ; 0 ;  0 ;  0 ;  0 ;  0 ;  5 ;  -5 ;  0 ;  0 ;  0 ;  0 ;  0 ;  0 ;  0 ;  0 ;  0 ;  0 ;  0 ;  0 ;  0 ;  5 ;  0 ;  0 ;  0 ;  0 ;  0 ;  0 ;  5 ;  0 ;  0 ;  0 ;  0 ;  0 ;  0 ;  5 ;  5 ;  5 ;  5 ;  5 ;  5 ;  0 ;  0 ;  0 ;  0 ;  0 ;  0 ;  0 ;  0 ;  0 ;  0 ;  0 ;  0 ;  0 ;  0 ;  0 ;  0 ' 
COUPSDIM6TOP[3]=' cQq81 ; 0 ;  0 ;  0 ;  0 ;  0 ;  0 ;  0 ;  5 ;  -5 ;  0 ;  0 ;  